In [2]:
import string
from datetime import date, datetime
from math import ceil
import re
import shutil
from pathlib import Path

import pandas as pd

In [2]:
def daily_words_calculate(word_goal, goal_start_date, goal_finish_date):
    # We currently assume you are starting at zero words, or are factoring your already existant words into your decision.
    # This assumption may be worth revisiting later
    #NOTE: changed incoming date format #days_left = abs((datetime.strptime(goal_finish_date,"%d/%m/%Y") - datetime.strptime(goal_start_date,"%d/%m/%Y")).days)
    days_left = abs((datetime.strptime(goal_finish_date,"%Y-%m-%d") - datetime.strptime(goal_start_date,"%Y-%m-%d")).days)
    daily_target = ceil(int(word_goal)/days_left)
    return daily_target


def word_goal_calculate(daily_target, goal_start_date, goal_finish_date):
    # We currently assume you are starting at zero words, or are factoring your already existant words into your decision.
    # This assumption may be worth revisiting later
    #NOTE: changed incoming date format #days_left = abs((datetime.strptime(goal_finish_date,"%d/%m/%Y") - datetime.strptime(goal_start_date,"%d/%m/%Y")).days)
    days_left = abs((datetime.strptime(goal_finish_date,"%Y-%m-%d") - datetime.strptime(goal_start_date,"%Y-%m-%d")).days)
    word_goal = int(daily_target)*days_left
    return word_goal

In [3]:
def change_goal(goal_start_date, goal_end_date, word_goal=None, daily_target=None):
    if (word_goal is not None) and (daily_target is None):
        goal = daily_words_calculate(word_goal, goal_start_date, goal_end_date)
        return goal
    elif (daily_target is not None) and (word_goal is None):
        goal = word_goal_calculate(daily_target, goal_start_date, goal_end_date)
        return goal
    else:
        raise Exception("Error: provide word_goal or daily_target, but not both.")

In [4]:
import sqlite3
sqlite3_path = './database/nanite_storage.sqlite3'

query = '''INSERT INTO words values (
    ?, ?, ?, ?, ?, ?) '''
params = [None, 1, 1, '2021-03-12 08:00:00.000000', 22000, 1000]
conn = sqlite3.connect(sqlite3_path)
cur = conn.cursor()
cur.execute(query, params)
conn.commit()

In [5]:
from collections import namedtuple
WordVals = namedtuple('Wordcounts', 'record_id project_id author_id Wdate Wcount Wtarget')

In [12]:
def streak_length(df_col):
    d_list = []
    for i in df_col:
            if i == 0:
                d = 0
            else:
                d += 1
            d_list.append(d)
    return d_list

def is_streak(i, count_col, target_col):
    
    if i == 0:
        return 0
    else: 
        pass
    if i == 0:
        pass
    elif (count_col.iloc[i] - count_col.iloc[i-1]) >= (target_col.iloc[i]): 
        return 1
    else:
        return 0

In [41]:
        conn = sqlite3.connect(sqlite3_path)
        cur = conn.cursor()
        cur.execute("SELECT * FROM words where project_id=?", (1,))
        row = cur.fetchall()
        data = [dict(WordVals(*row[i])._asdict()) for i in range(len(row))]
        conn.close()
        df = pd.DataFrame(data)
        df.index = pd.DatetimeIndex(df['Wdate'])

        earliest = df.index[0]
        latest = df.index[-1]

        new_index = pd.date_range(earliest, latest, freq='D')

        df = df.groupby(pd.Grouper(freq='D')).last()
        df = df.reindex(new_index, method='ffill')
        df = df.fillna(method='ffill')
        df['Wtarget_sum'] = df['Wtarget'].cumsum()
        df['Wdate'] = df['Wdate'].apply(lambda x: x[:10])
        df['daily_words'] = [(df[['Wcount']].iloc[i] - df[['Wcount']].iloc[i-1]).clip(0) for i in range(len(df))]
        df['streak'] = [is_streak(i, df['Wcount'], df['Wtarget']) for i in range(len(df))]
        d_list = streak_length(df['streak'])
        df['streak'] = pd.Series(d_list, index=new_index)
        
        df = df.groupby(pd.Grouper(freq='D')).last()
        records = df.to_dict(orient='records')
  


In [42]:
df

,Wcount,Wdate,Wtarget,author_id,project_id,record_id,Wtarget_sum,daily_words,streak
2021-03-01,1500.0,2021-03-01,1000.0,1.0,1.0,1.0,1000.0,0.0,0
2021-03-02,2300.0,2021-03-02,1000.0,1.0,1.0,2.0,2000.0,800.0,0
2021-03-03,2900.0,2021-03-03,1000.0,1.0,1.0,3.0,3000.0,600.0,0
2021-03-04,3500.0,2021-03-04,1000.0,1.0,1.0,4.0,4000.0,600.0,0
2021-03-05,3500.0,2021-03-04,1000.0,1.0,1.0,4.0,5000.0,0.0,0
2021-03-06,5200.0,2021-03-06,1000.0,1.0,1.0,5.0,6000.0,1700.0,1
2021-03-07,6500.0,2021-03-07,1000.0,1.0,1.0,6.0,7000.0,1300.0,2
2021-03-08,6500.0,2021-03-07,1000.0,1.0,1.0,6.0,8000.0,0.0,0
2021-03-09,6500.0,2021-03-07,1000.0,1.0,1.0,6.0,9000.0,0.0,0
2021-03-10,6500.0,2021-03-07,1000.0,1.0,1.0,6.0,10000.0,0.0,0


In [29]:
[print() for i in range(len(df))]

TypeError: cannot do index indexing on <class 'pandas.core.indexes.datetimes.DatetimeIndex'> with these indexers [0] of <class 'int'>

[Wcount    0.0
 dtype: float64,
 Wcount    800.0
 dtype: float64,
 Wcount    600.0
 dtype: float64,
 Wcount    600.0
 dtype: float64,
 Wcount    0.0
 dtype: float64,
 Wcount    1700.0
 dtype: float64,
 Wcount    1300.0
 dtype: float64,
 Wcount    0.0
 dtype: float64,
 Wcount    0.0
 dtype: float64,
 Wcount    0.0
 dtype: float64,
 Wcount    0.0
 dtype: float64,
 Wcount    15500.0
 dtype: float64]

In [19]:
def 

[{'Wcount': 1500.0,
  'Wdate': '2021-03-01',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 1.0,
  'Wtarget_sum': 1000.0,
  'streak': 0},
 {'Wcount': 2300.0,
  'Wdate': '2021-03-02',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 2.0,
  'Wtarget_sum': 2000.0,
  'streak': 0},
 {'Wcount': 2900.0,
  'Wdate': '2021-03-03',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 3.0,
  'Wtarget_sum': 3000.0,
  'streak': 0},
 {'Wcount': 3500.0,
  'Wdate': '2021-03-04',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 4.0,
  'Wtarget_sum': 4000.0,
  'streak': 0},
 {'Wcount': 3500.0,
  'Wdate': '2021-03-04',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 4.0,
  'Wtarget_sum': 5000.0,
  'streak': 0},
 {'Wcount': 5200.0,
  'Wdate': '2021-03-06',
  'Wtarget': 1000.0,
  'author_id': 1.0,
  'project_id': 1.0,
  'record_id': 5.0,
  'Wtarget_sum': 6000.0,
  'strea

2

In [9]:
def word_streaker(df, c, i):
    if i == 0:
        c = 0
    elif (df['Wcount'].iloc[i] - df['Wcount'].iloc[i-1]) > (df['Wtarget']):
        c = c + 1
    else:
        c = 0
    return c 

In [ ]:
        """
        df = df.groupby(pd.Grouper(freq='D')).last()
        records = df.to_dict(orient='records')
        return records
        """